# Machine Learning Engineer Nanodegree

# Capstone: Predicting Loan Defaults In Peer-To-Peer Lending

## I. Getting Started
In this project, we will analyze a dataset containing data on potential borrowers. The goal of this project is to build a model predicting the loan default of potential borrowers. 

The dataset for this project can be found on [Lending Club](https://www.lendingclub.com/info/download-data.action).

Run the code block below to load the wholesale customers dataset, along with a few of the necessary Python libraries required for this project. You will know the dataset loaded successfully if the size of the dataset is reported.


In [11]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display # Allows the use of display() for DataFrames

# Pretty display for notebooks
%matplotlib inline
plt.style.use('fivethirtyeight')

# Load the accepted loan dataset 
# low_memory and skiprows in read_csv because the file is big
try:
    loan_data = pd.read_csv("LoanStats3a.csv", low_memory = False, skiprows = 1)
    print "The loan dataset has {} samples with {} features each.".format(*loan_data.shape)
except:
    print "The loan dataset could not be loaded. Is the dataset missing?"

The loan dataset has 42538 samples with 111 features each.


## II. Analysis

## Introduction To The Data

(explain the process of Lending club loan approval)

The dictionary data file is provided with the project in order to refer to it later in our data exploration. This contains information about the various columns and will be useful when we clean up the dataset. The data being used is the data from 2007 to 2011 mostly because when can be almost certain that all the loans have been either repaid or defaulted upon.

In [12]:
half_count = len(loan_data) / 2
loan_data = loan_data.dropna(thresh=half_count, axis=1)
loan_data = loan_data.drop(['desc', 'url'],axis=1)
loan_data.to_csv('loans_2007.csv', index=False)

In [15]:
loans_2007 = pd.read_csv('loans_2007.csv', low_memory = False)
loans_2007.drop_duplicates()

loans_2007.iloc[0]

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

In [16]:
loans_2007.shape[1]

52

The Dataframe is cumbersome and we had to set the `low_memory` to `False` to avoid a warning message from the notebook. This is due to the numerous columns of the dataset. Let us explore the dataset with the data dictionary this will be useful as we go through the data and try to clean it.

We will need to be careful about data from the future, this type of leakage could throw off the useful predictions of our model. A clean example is information about the borrower after the loan was approved, this is not data that we would have at our disposal. 

We will be splitting the columns in 4 giving us 13 features to analysis and try to make sense of. This part is crucial in order to understand the data and avoid error while fitting our machine learning model later on. 


### Build A Table To Analyse And Visualize
We will build a table with 2 csv files. We will use the first entry of the `loans_2007.csv` file to explore the meaning of the 52 columns. 

In [30]:
first_entry = loans_2007.iloc[0]
first_entry.to_csv('first_entry.csv', index = True)

In [29]:
description = pd.read_csv('LCDataDictionary.csv')

description.shape

(115, 2)

In [52]:
import csv
list_first_entry = open('first_entry.csv', 'r')
first_csvreader = csv.reader(list_first_entry)
first_list = list(first_csvreader)

list_data_dictio = open('LCDataDictionary.csv', 'r')
second_csvreader = csv.reader(list_data_dictio)
second_list = list(second_csvreader)

table = []
for row in first_list:
    table.append(row[0])

new_table = []
for col in second_list:
    if col[0] in table:
        new_table.append(col)

### First Set Of Features

### Second Set Of Features

### Third Set Of Features

### Fourth Set Of Features

### Target Column

## III. Methodology

### Preparing The Features